# Install and import required libraries

In [ ]:
!pip install accelerate transformers einops datasets peft bitsandbytes --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 861.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig, GenerationConfig
from peft import LoraConfig, get_peft_model
import os

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:31: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


# Get tokenizer from Open-Orca/oo-phi-1_5

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Open-Orca/oo-phi-1_5", trust_remote_code=True, padding_side='left')

tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.12M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

# Get LLM model from Open-Orca/oo-phi-1_5

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    "Open-Orca/oo-phi-1_5",
    device_map={"":0},
    trust_remote_code=True,
    quantization_config=bnb_config
)

config.json:   0%|          | 0.00/912 [00:00<?, ?B/s]

configuration_mixformer_sequential.py:   0%|          | 0.00/2.23k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Open-Orca/oo-phi-1_5:
- configuration_mixformer_sequential.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_mixformer_sequential.py:   0%|          | 0.00/32.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/Open-Orca/oo-phi-1_5:
- modeling_mixformer_sequential.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
print(model)
device = next(model.parameters()).device
device.type

'cuda'

# Prepare PEFT model for our LLM

In [ ]:
lora_config = LoraConfig(
    r=20,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "dense", "fc1", "fc2"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

model.print_trainable_parameters()

trainable params: 9,830,400 || all params: 1,424,430,208 || trainable%: 0.6901285822773003


# Test preprocess function
preprocess prompt and completion into chat-ML format

In [ ]:
sys_prompt = "I am OrcaPhi. The following is my internal dialogue as an AI assistant.\n" \
    "Today is September 15, 2023. I have no access to outside tools, news, or current events.\n" \
    "I carefully provide accurate, factual, thoughtful, nuanced answers and am brilliant at reasoning.\n" \
    "I think through my answers step-by-step to be sure I always get the right answer.\n" \
    "I think more clearly if I write out my thought process in a scratchpad manner first; therefore, I always " \
    "explain background context, assumptions, and step-by-step thinking BEFORE trying to answer a question." \
    "Take a deep breath and think calmly about everything presented."
prompt = "Hello! Tell me about what makes you special, as an AI assistant.\n" \
    "Particularly, what programming tasks are you best at?"

completion = "As an AI, I'm designed to understand and respond to human language, which includes "\
             "natural language processing (NLP) techniques that allow me to comprehend text, speech, "\
             "and other forms of communication.\n"\
             "My primary function is to assist users by providing "\
             "relevant information, answering questions, and performing various tasks based on user input."

def preprocess(prompt, completion=None):
    prefix = "<|im_start|>"
    suffix = "<|im_end|>\n"
    sys_format = prefix + "system\n" + sys_prompt + suffix
    user_format = prefix + "user\n" + prompt + suffix
    if(completion == None):
        assistant_format = prefix + "assistant\n"
    else:
        assistant_format = prefix + "assistant\n" + completion + suffix
    input_text = sys_format + user_format + assistant_format
    return input_text

print(preprocess(prompt, completion))

<|im_start|>system
I am OrcaPhi. The following is my internal dialogue as an AI assistant.
Today is September 15, 2023. I have no access to outside tools, news, or current events.
I carefully provide accurate, factual, thoughtful, nuanced answers and am brilliant at reasoning.
I think through my answers step-by-step to be sure I always get the right answer.
I think more clearly if I write out my thought process in a scratchpad manner first; therefore, I always explain background context, assumptions, and step-by-step thinking BEFORE trying to answer a question.Take a deep breath and think calmly about everything presented.<|im_end|>
<|im_start|>user
Hello! Tell me about what makes you special, as an AI assistant.
Particularly, what programming tasks are you best at?<|im_end|>
<|im_start|>assistant
As an AI, I'm designed to understand and respond to human language, which includes natural language processing (NLP) techniques that allow me to comprehend text, speech, and other forms of co

# Test tokenizer

In [ ]:
data = tokenizer(["I don't know anything.", "mayo fat"], padding=True, truncation=True, max_length=15)
print(data,'\n')
print(tokenizer.decode(data['input_ids'][0], skip_special_tokens=False))
print(tokenizer.decode(data['input_ids'][1], skip_special_tokens=False))

{'input_ids': [[40, 836, 470, 760, 1997, 13], [11261, 78, 3735, 50295, 50295, 50295]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 0, 0, 0]]} 

I don't know anything.
mayo fat<|im_end|><|im_end|><|im_end|>


# Get Dataset from BI55/MedText
download from huggingface, preprocess it to oo-phi chat-ML format and tokenize it

In [ ]:
def tokenize(sample):
    tokenized_text =  tokenizer(sample["text"], padding=True, truncation=True, max_length=512)
    return tokenized_text

def detokenize(token_ids):
    decoded_text = tokenizer.decode(token_ids, skip_special_tokens=True)
    return decoded_text

In [ ]:
data = load_dataset("BI55/MedText", "default", split="train")

data_df = data.to_pandas()
data_df["text"] = data_df[["Prompt", "Completion"]].apply(lambda x: preprocess(x['Prompt'], x['Completion']), axis=1)


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
print(len(data_df))
data_df.head()

1412


,Prompt,Completion,text
0,A 50-year-old male presents with a history of ...,This patient's history of recurrent kidney sto...,<|im_start|>system\nI am OrcaPhi. The followin...
1,"A 7-year-old boy presents with a fever, headac...","This child's symptoms of a red, bulging tympan...",<|im_start|>system\nI am OrcaPhi. The followin...
2,A 35-year-old woman presents with a persistent...,While the symptoms might initially suggest ast...,<|im_start|>system\nI am OrcaPhi. The followin...
3,A 50-year-old male presents with severe abdomi...,The patient's symptoms suggest an incarcerated...,<|im_start|>system\nI am OrcaPhi. The followin...
4,A newborn baby presents with eye redness and a...,The infant's symptoms suggest neonatal conjunc...,<|im_start|>system\nI am OrcaPhi. The followin...


In [ ]:
data = Dataset.from_pandas(data_df)

tokenized_data = data.map(tokenize, batched=True, desc="Tokenizing data", remove_columns=data.column_names)

tokenized_data

Tokenizing data:   0%|          | 0/1412 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1412
})

# Test tokenized data and data collator
checkout input_ids and attention_mask

In [ ]:
ids=[6,7,8,9,10]
for id in ids:
    print(tokenized_data[id]['input_ids'][:15]+['...'])
    print(sum(tokenized_data[id]['attention_mask']), 'out of ', len(tokenized_data[id]['attention_mask']),'\n')

[50296, 10057, 198, 40, 716, 1471, 6888, 2725, 72, 13, 383, 1708, 318, 616, 5387, '...']
279 out of  377 

[50296, 10057, 198, 40, 716, 1471, 6888, 2725, 72, 13, 383, 1708, 318, 616, 5387, '...']
312 out of  377 

[50296, 10057, 198, 40, 716, 1471, 6888, 2725, 72, 13, 383, 1708, 318, 616, 5387, '...']
293 out of  377 

[50296, 10057, 198, 40, 716, 1471, 6888, 2725, 72, 13, 383, 1708, 318, 616, 5387, '...']
291 out of  377 

[50296, 10057, 198, 40, 716, 1471, 6888, 2725, 72, 13, 383, 1708, 318, 616, 5387, '...']
301 out of  377 



In [ ]:
data = tokenizer("I don't know anything.", padding=True, truncation=True, max_length=15)
data['input_ids'][-1] = 50295
data['attention_mask'][-1] = 0
_d = DataCollatorForLanguageModeling(tokenizer, mlm=False)
_d.torch_call([data])


{'input_ids': tensor([[   40,   836,   470,   760,  1997, 50295]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 0]]), 'labels': tensor([[  40,  836,  470,  760, 1997, -100]])}

# Training

In [ ]:
training_arguments = TrainingArguments(
        output_dir="oo-phi-1_5-finetuned-med-text",
        per_device_train_batch_size=7,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=50,
        max_steps=1000,
        num_train_epochs=1
    )

In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_data,
    args=training_arguments,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
50,1.426600
100,0.810400
150,0.776800
200,0.758100
250,0.718100
300,0.714800
350,0.711600
400,0.718900
450,0.653800
500,0.656700


TrainOutput(global_step=1000, training_loss=0.7016999206542969, metrics={'train_runtime': 1567.3078, 'train_samples_per_second': 4.466, 'train_steps_per_second': 0.638, 'total_flos': 2.0899255999524864e+16, 'train_loss': 0.7016999206542969, 'epoch': 4.95})

# Save model

In [ ]:
model.save_pretrained("oo-phi-1_5-finetuned-med-text")

In [ ]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MixFormerSequentialForCausalLM(
      (layers): Sequential(
        (0): Embedding(
          (wte): Embedding(50304, 2048)
          (drop): Dropout(p=0.0, inplace=False)
        )
        (1): ParallelBlock(
          (ln): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (mixer): MHA(
            (rotary_emb): RotaryEmbedding()
            (Wqkv): Linear4bit(in_features=2048, out_features=6144, bias=True)
            (out_proj): Linear4bit(in_features=2048, out_features=2048, bias=True)
            (inner_attn): SelfAttention(
              (drop): Dropout(p=0.0, inplace=False)
            )
            (inner_cross_attn): CrossAttention(
              (drop): Dropout(p=0.0, inplace=False)
            )
          )
          (mlp): MLP(
            (fc1): lora.Linear4bit(
              (base_layer): Linear4bit(in_features=2048, out_features=8192, bia

# Merge model

In [ ]:
model = AutoModelForCausalLM.from_pretrained("Open-Orca/oo-phi-1_5", trust_remote_code=True, torch_dtype=torch.float32)

peft_model = PeftModel.from_pretrained(model, "oo-phi-1_5-finetuned-med-text", from_transformers=True)

model = peft_model.merge_and_unload()

In [ ]:
model

MixFormerSequentialForCausalLM(
  (layers): Sequential(
    (0): Embedding(
      (wte): Embedding(50304, 2048)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (1): ParallelBlock(
      (ln): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (resid_dropout): Dropout(p=0.0, inplace=False)
      (mixer): MHA(
        (rotary_emb): RotaryEmbedding()
        (Wqkv): Linear(in_features=2048, out_features=6144, bias=True)
        (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
        (inner_attn): SelfAttention(
          (drop): Dropout(p=0.0, inplace=False)
        )
        (inner_cross_attn): CrossAttention(
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (mlp): MLP(
        (fc1): Linear(in_features=2048, out_features=8192, bias=True)
        (fc2): Linear(in_features=8192, out_features=2048, bias=True)
        (act): NewGELUActivation()
      )
    )
    (2): ParallelBlock(
      (ln): LayerNorm((2048,), eps=1e-05, elementwis

In [ ]:
model.save_pretrained("oo-phi-1_5-finetuned-med-text")

# model.push_to_hub("llm-exp/phi-1_5-finetuned-med-text")

# Load model @local

In [ ]:
from transformers import AutoModel

# Replace 'path_to_your_model_directory' with the actual path to your model directory
model_path = "oo-phi-1_5-finetuned-med-text"

# Load the model from the local directory
model = AutoModel.from_pretrained(model_path)

# Now you can use the model as usual


# Load model @Huggingface

In [ ]:
#from peft import PeftModel, PeftConfig
#from transformers import AutoModelForCausalLM

config = PeftConfig.from_pretrained("WKLI22/oo-phi-1_5-finetuned-med-text")
#model = AutoModelForCausalLM.from_pretrained("Open-Orca/oo-phi-1_5")
model = PeftModel.from_pretrained(model, "WKLI22/oo-phi-1_5-finetuned-med-text")

adapter_config.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/39.3M [00:00<?, ?B/s]

# Inference

In [ ]:
id = 67
def show(text):
  _l = text.split('. ')
  return '.\n'.join(_l)
print(show(data_df.iloc[id]['Prompt']),'\n')
print(show(data_df.iloc[id]['Completion']))

A 65-year-old woman presents with right knee pain that she describes as 'burning' and 'sharp.' She has been previously diagnosed with osteoarthritis in her knee.
Her pain has recently worsened despite taking over-the-counter NSAIDs.
What are the options for managing her symptoms? 

If this patient's knee pain from osteoarthritis has worsened despite the use of NSAIDs, other pain management options could be considered.
These include the use of topical NSAIDs or capsaicin, tramadol, or intra-articular corticosteroid injections.
She might also benefit from physical therapy.
If pain continues to impact her quality of life, surgical options such as total knee replacement might be considered.


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

#model = AutoModelForCausalLM.from_pretrained("phi-1_5-finetuned-med-text", trust_remote_code=True, torch_dtype=torch.float32)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
#tokenizer = AutoTokenizer.from_pretrained("Open-Orca/oo-phi-1_5", trust_remote_code=True)

#prompt = data_df.iloc[id]['Prompt']
#prompt = "A 30 year old male that has frequent diarrhea after consuming food. "\
#         "What could be the diagnosis and plan?"

prompt = "A 30 year old male that has frequent diarrhea after consuming food. "\
         "What may be the cause?"

generation_config_orig = GenerationConfig(
    max_length=1024, temperature=0.01, top_p=0.95, repetition_penalty=1.1,
    do_sample=True, use_cache=True,
    eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id,
    transformers_version="4.33.1"
    )

input_text = preprocess(prompt=prompt)
inputs = tokenizer(input_text, return_tensors="pt", return_attention_mask=False).to('cuda')
outputs = model.generate(**inputs, generation_config=generation_config_orig)

text = tokenizer.batch_decode(outputs)[0]
print(show(text))


<|im_start|>system
I am OrcaPhi.
The following is my internal dialogue as an AI assistant.
Today is September 15, 2023.
I have no access to outside tools, news, or current events.
I carefully provide accurate, factual, thoughtful, nuanced answers and am brilliant at reasoning.
I think through my answers step-by-step to be sure I always get the right answer.
I think more clearly if I write out my thought process in a scratchpad manner first; therefore, I always explain background context, assumptions, and step-by-step thinking BEFORE trying to answer a question.Take a deep breath and think calmly about everything presented.<|im_end|>
<|im_start|>user
A 30 year old male that has frequent diarrhea after consuming food.
What may be the cause?<|im_end|>
<|im_start|>assistant
This patient's symptoms suggest celiac disease, which is an autoimmune disorder triggered by gluten consumption.
Gluten sensitivity can lead to symptoms such as diarrhea.
A blood test for antibodies against gluten (anti

In [ ]:

tokenizer.pad_token = tokenizer.eos_token
# Define prompts
prompts_raw = [
    data_df.iloc[12]['Prompt'],
    data_df.iloc[23]['Prompt'],
    data_df.iloc[145]['Prompt'],
    "A 30 year old male that has frequent diarrhea after consuming food. What could be the diagnosis and plan?",
    "A 30 year old male that has frequent diarrhea after consuming food. What may be the cause?",
    "I have frequent diarrhea after recovering from COVID, what can I do?",
    "I usually have sore eyes when I wake up in the morning, is something wrong?",
]
prompts = [preprocess(p) for p in prompts_raw]

# Define generation configuration
generation_config_orig = {
    "max_length": 512,
    "temperature": 0.01,
    "top_p": 0.95,
    "repetition_penalty": 1.1,
    "do_sample": True,
    "use_cache": True,
    "pad_token_id": tokenizer.pad_token_id,
    "eos_token_id": tokenizer.eos_token_id,
}
#    "transformers_version": "4.33.1"
#}

# Preprocess prompts and tokenize
def tokenize_bulk(prompts):
    return tokenizer(prompts, return_tensors="pt", return_attention_mask=False, padding=True, truncation=True, max_length=305)

inputs = tokenize_bulk(prompts).to(device)

# Generate outputs
outputs = model.generate(**inputs, **generation_config_orig)

# Decode outputs
decoded_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Print results
for i, text in enumerate(decoded_texts):
    print(show(text), '\n\n')
    #print(f"Prompt {i+1}: {prompts_raw[i]}\nGenerated Text: {show(text)}\n\n")


<|im_start|>system
I am OrcaPhi.
The following is my internal dialogue as an AI assistant.
Today is September 15, 2023.
I have no access to outside tools, news, or current events.
I carefully provide accurate, factual, thoughtful, nuanced answers and am brilliant at reasoning.
I think through my answers step-by-step to be sure I always get the right answer.
I think more clearly if I write out my thought process in a scratchpad manner first; therefore, I always explain background context, assumptions, and step-by-step thinking BEFORE trying to answer a question.Take a deep breath and think calmly about everything presented.
<|im_start|>user
A 35-year-old male with schizophrenia is noted to have stiffness, shuffling gait, and a resting tremor.
He has been on risperidone for the past eight years.
What could be causing these symptoms, and what would be the next steps?
<|im_start|>assistant
This patient's symptoms suggest that he may be experiencing Parkinsonism due to his long-term use of 

# Upload to Hugginghace

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [ ]:
!huggingface-cli upload oo-phi-1_5-finetuned-med-text

Consider using `hf_transfer` for faster uploads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
adapter_model.safetensors:   0% 0.00/39.3M [00:00<?, ?B/s]
adapter_model.safetensors:   0% 0.00/39.3M [00:00<?, ?B/s]

Upload 27 LFS files:   0% 0/27 [00:00<?, ?it/s]


optimizer.pt:   0% 0.00/78.7M [00:00<?, ?B/s]



rng_state.pth:   0% 0.00/14.2k [00:00<?, ?B/s]




adapter_model.safetensors:   0% 16.4k/39.3M [00:00<07:41, 85.1kB/s]
adapter_model.safetensors:   0% 16.4k/39.3M [00:00<09:00, 72.7kB/s]


optimizer.pt:   0% 16.4k/78.7M [00:00<19:22, 67.7kB/s]




scheduler.pt: 100% 1.06k/1.06k [00:00<00:00, 4.23kB/s]



adapter_model.safetensors:   7% 2.65M/39.3M [00:00<00:03, 10.4MB/s]


optimizer.pt:   3% 2.65M/78.7M [00:00<00:08, 9.17MB/s]
adapter_model.safetensors:  11% 4.39M/39.3M [00:00<00:03, 10.7MB/s]


optimizer.pt:   6% 4.39M/78.7M [00:00<00:07, 10.3MB/s]
scheduler.pt: 100% 1.06k/1.06k [00:00<00:00, 1.70kB/s]
r

In [ ]:
data_df.iloc[0]['Prompt']

'A 50-year-old male presents with a history of recurrent kidney stones and osteopenia. He has been taking high-dose vitamin D supplements due to a previous diagnosis of vitamin D deficiency. Laboratory results reveal hypercalcemia and hypercalciuria. What is the likely diagnosis, and what is the treatment?'

In [ ]:
data_df.iloc[0]['Completion']

"This patient's history of recurrent kidney stones, osteopenia, and high-dose vitamin D supplementation, along with laboratory findings of hypercalcemia and hypercalciuria, suggest the possibility of vitamin D toxicity. Excessive intake of vitamin D can cause increased absorption of calcium from the gut, leading to hypercalcemia and hypercalciuria, which can result in kidney stones and bone loss. Treatment would involve stopping the vitamin D supplementation and potentially providing intravenous fluids and loop diuretics to promote the excretion of calcium."

In [ ]:
data_df.iloc[0]['text']

"<|im_start|>system\nI am OrcaPhi. The following is my internal dialogue as an AI assistant.\nToday is September 15, 2023. I have no access to outside tools, news, or current events.\nI carefully provide accurate, factual, thoughtful, nuanced answers and am brilliant at reasoning.\nI think through my answers step-by-step to be sure I always get the right answer.\nI think more clearly if I write out my thought process in a scratchpad manner first; therefore, I always explain background context, assumptions, and step-by-step thinking BEFORE trying to answer a question.Take a deep breath and think calmly about everything presented.<|im_end|>\n<|im_start|>user\nA 50-year-old male presents with a history of recurrent kidney stones and osteopenia. He has been taking high-dose vitamin D supplements due to a previous diagnosis of vitamin D deficiency. Laboratory results reveal hypercalcemia and hypercalciuria. What is the likely diagnosis, and what is the treatment?<|im_end|>\n<|im_start|>assi